# Plotly anaylsis 

In [40]:
!pip3 freeze

absl-py==1.4.0
anyio==3.6.2
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arrow==1.2.3
astcheck==0.3.0
asttokens==2.2.1
astunparse==1.6.3
attrs==23.1.0
backcall==0.2.0
beautifulsoup4==4.12.2
bleach==6.0.0
bokeh==3.1.1
cachetools==5.3.1
certifi==2023.5.7
cffi==1.15.1
charset-normalizer==3.1.0
-e git+https://github.com/newaetech/chipwhisperer@a03e37ba6cdcb201c2d9d9d4d86a17dcc2f699f1#egg=chipwhisperer
click==8.1.3
cloudpickle==2.2.1
colorcet==3.0.1
comm==0.1.3
configobj==5.0.8
contourpy==1.0.7
cycler==0.11.0
Cython==0.29.35
dask==2023.5.1
datashader==0.14.4
datashape==0.5.2
debugpy==1.6.7
decorator==5.1.1
defusedxml==0.7.1
ECPy==1.2.5
executing==1.2.0
fastdtw==0.3.4
fastjsonschema==2.17.1
flatbuffers==23.5.26
fonttools==4.39.4
fqdn==1.5.1
fsspec==2023.5.0
gast==0.4.0
google-auth==2.22.0
google-auth-oauthlib==1.0.0
google-pasta==0.2.0
graphviz==0.20.1
grpcio==1.56.2
h5py==3.9.0
holoviews==1.16.0
idna==3.4
importlib-metadata==6.6.0
importlib-resources==5.12.0
ipykernel==6.23.1
ipython==8

In [38]:
#!pip install plotly==5.15.0
!pip install scikit-learn

     |████████████████████████████████| 10.9 MB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 302 kB 39.5 MB/s eta 0:00:01
You should consider upgrading via the '/home/manoj/.pyenv/versions/3.9.5/envs/cw/bin/python3.9 -m pip install --upgrade pip' command.


In [ ]:
import plotly.express as px
fig = px.bar(x=["a", "b", "c"], y=[1, 3, 2])
fig.write_html('img/first_figure.html', auto_open=True)

In [1]:
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
import numpy as np
import chipwhisperer as cw


In [2]:
project = cw.open_project("/mnt/hgfs/trace_key_f4_lat0_30db/key_90.cwp")
#project = cw.open_project("traces/checkTraces1.cwp")
print(project.location)
xShape = project.traces[0].wave.shape[0]
x=np.linspace(0,xShape-1, xShape)
print(x)
colorList = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00"]

fig = go.Figure()
fig = make_subplots(rows=1, cols=1, shared_yaxes=True, shared_xaxes=True, \
)
for idx in range(5):
    fig.add_trace(go.Line(x=x, y=project.traces[idx].wave, line=dict(color=colorList[idx])), row=1, col=1)

fig.update_layout(
    xaxis1_title="Time",
    yaxis1_title="Power",
    showlegend=True
  )

savePath = "img/f415_key90_trace_key.html"
fig.write_html(savePath, auto_open=True)


/mnt/hgfs/trace_key_f4_lat0_30db
[0.000e+00 1.000e+00 2.000e+00 ... 1.497e+03 1.498e+03 1.499e+03]


/home/manoj/.pyenv/versions/cw/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




# Low Pass Filter to remove noise from F415 signals

In [3]:
from scipy import signal

In [21]:
fs = 22050.0       # Sample rate, Hz
##Frequency for cut-off
cutoff = 2600.0   # Desired cutoff frequency, Hz
trans_width = 100  # Width of transition from pass band to stop band, Hz
##numtaps if low will give plots with troughs, need tobe fairly high for a good response
numtaps = 64     # Size of the FIR filter.
taps = signal.remez(numtaps, [0, cutoff, cutoff + trans_width, 0.5*fs], [1, 0], fs=fs)




In [39]:
fs = 7.38e6       # Sample rate, Hz
##Frequency for cut-off
cutoff = 4.92e5    # Desired cutoff frequency, Hz
trans_width = 100  # Width of transition from pass band to stop band, Hz
##numtaps if low will give plots with troughs, need tobe fairly high for a good response
numtaps = 64     # Size of the FIR filter.
taps = signal.remez(numtaps, [0, cutoff, cutoff + trans_width, 0.5*fs], [1, 0], fs=fs)

In [40]:
project = cw.open_project("/mnt/hgfs/trace_key_f4_lat0_30db/key_90.cwp")
#project = cw.open_project("traces/checkTraces1.cwp")
print(project.location)
x_raw = [project.traces[idx].wave for idx in range(5)]
x_raw

/mnt/hgfs/trace_key_f4_lat0_30db


[memmap([0.0703125 , 0.03125   , 0.05957031, ..., 0.01953125, 0.00585938,
         0.01171875]),
 memmap([ 0.05761719,  0.02050781,  0.05957031, ...,  0.01855469,
         -0.00097656,  0.00097656]),
 memmap([0.06347656, 0.015625  , 0.06347656, ..., 0.02734375, 0.00097656,
         0.02148438]),
 memmap([ 0.06640625,  0.03125   ,  0.0625    , ...,  0.015625  ,
         -0.00292969,  0.00488281]),
 memmap([0.06640625, 0.03320312, 0.06054688, ..., 0.02246094, 0.0078125 ,
         0.01269531])]

In [41]:
x_lpf = signal.lfilter(taps, [1], x_raw)
x_lpf

array([[ 0.01440304,  0.00055383,  0.00458223, ...,  0.00297021,
         0.00106371,  0.01007299],
       [ 0.01180249, -0.00059083,  0.00638225, ...,  0.00012991,
         0.00063232,  0.01322219],
       [ 0.01300275, -0.00207834,  0.00717005, ...,  0.00060158,
        -0.00429094,  0.01041499],
       [ 0.01360287,  0.00087869,  0.00546133, ..., -0.00041533,
        -0.00277371,  0.00831256],
       [ 0.01360287,  0.00127878,  0.00489881, ...,  0.0007752 ,
        -0.00069798,  0.00788181]])

In [42]:
xShape = x_lpf[0].shape[0]
x=np.linspace(0,xShape-1, xShape)
print(x)
colorList = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00"]

fig = go.Figure()
fig = make_subplots(rows=2, cols=1, shared_yaxes=True, shared_xaxes=True, \
)
for idx in range(5):
    fig.add_trace(go.Line(x=x, y=x_raw[idx], line=dict(color=colorList[idx])), row=1, col=1)
    fig.add_trace(go.Line(x=x, y=x_lpf[idx], line=dict(color=colorList[idx])), row=2, col=1)

fig.update_layout(
    xaxis1_title="Time",
    yaxis1_title="Power",
    showlegend=True
  )

savePath = "img/f415_key90_lpf.html"
fig.write_html(savePath, auto_open=True)

[0.000e+00 1.000e+00 2.000e+00 ... 1.497e+03 1.498e+03 1.499e+03]


/home/manoj/.pyenv/versions/cw/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




## Plot freqency response for before and after low pass filter

In [25]:
from scipy.fft import fft, fftfreq, rfft, rfftfreq, ifft

In [43]:
fft_raw = rfft(x_raw)
fft_lpf = rfft(x_lpf)

In [44]:
samplingRate = 7.38e6
numSamples = 1500
xf = rfftfreq(numSamples, 1 / samplingRate)

In [45]:
colorList = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00"]

fig = go.Figure()
fig = make_subplots(rows=2, cols=1, shared_yaxes=True, shared_xaxes=True, \
)
for idx in range(5):
    fig.add_trace(go.Line(x=xf, y=np.abs(fft_raw[idx]), line=dict(color=colorList[idx])), row=1, col=1)
    fig.add_trace(go.Line(x=xf, y=np.abs(fft_lpf[idx]), line=dict(color=colorList[idx])), row=2, col=1)

fig.update_layout(
    xaxis1_title="Time",
    yaxis1_title="Power",
    showlegend=True
  )

savePath = "img/f415_key90_fft_post_lpf.html"
fig.write_html(savePath, auto_open=True)

/home/manoj/.pyenv/versions/cw/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




## Median filtering to remove spikes

In [58]:
import scipy.ndimage
def median_filter(num):
    for _ in range(num):
        x_median = scipy.ndimage.median_filter(x_raw[0], 5)
    return x_median

In [56]:
median_filter(1)

array([0.04882812, 0.04882812, 0.04882812, ..., 0.03417969, 0.03417969,
       0.03417969])

In [59]:
xShape = x_raw[0].shape[0]
x=np.linspace(0,xShape-1, xShape)

colorList = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00"]

fig = go.Figure()
fig = make_subplots(rows=1, cols=1, shared_yaxes=True, shared_xaxes=True, \
)
fig.add_trace(go.Line(x=x, y=x_raw[0], line=dict(color=colorList[0]), name="Raw Signal"), row=1, col=1)
for idx in range(4):
    fig.add_trace(go.Line(x=x, y=median_filter(idx+1), line=dict(color=colorList[idx+1]), name="Median " + str(idx)), row=1, col=1)

fig.update_layout(
    xaxis1_title="Time",
    yaxis1_title="Power",
    showlegend=True
  )

savePath = "img/f415_key90_median_filter.html"
fig.write_html(savePath, auto_open=True)

/home/manoj/.pyenv/versions/cw/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




**Combine 2 sets of traces**

In [4]:
import pandas as pd
import numpy as np

project1 = cw.open_project("traces/f303_mbedtls_x4_key0.cwp")
project2 = cw.open_project("/mnt/hgfs/trace_key_f303_10kAdd/key_0.cwp")
print(project.location)
traceList=[]
keyList = []
df = pd.DataFrame(columns=['trace', 'key'])

for idx, (trace1, trace2) in enumerate(zip(project1.traces,project2.traces)):
    #print(f"trace1= {trace1.wave.shape}, trace2= {trace2.wave.shape}")
    traceList.append(np.concatenate((trace1.wave,trace2.wave), axis=0))
    keyList.append(trace1.key)

df["trace"] = traceList
df["key"] = keyList

#print(f"fullData shape= {len(traceList)}\ndf= {df.head}\ndf_1 = {df.iloc[0].shape}")

xShape = df.trace.iloc[0].shape[0]
x=np.linspace(0,xShape-1, xShape)

colorList = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00"]

fig = go.Figure()
fig = make_subplots(rows=1, cols=1, shared_yaxes=True, shared_xaxes=True, \
)
for idx in range(5):
    fig.add_trace(go.Line(x=x, y=df["trace"].iloc[idx], line=dict(color=colorList[idx])), row=1, col=1)

fig.update_layout(
    xaxis1_title="Time",
    yaxis1_title="Power",
    showlegend=True
  )

savePath = "img/f303_tinyaes_x4_full.html"
fig.write_html(savePath, auto_open=True)

/home/manoj/Documents/chipwhisperer/jupyter/courses/sca201/traces


/home/manoj/.pyenv/versions/cw/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




# Compare 7.38MHz and 4 MHz clock sampled traces

In [11]:
project1 = cw.open_project("/mnt/hgfs/trace_key_f303_10kAdd/key_0.cwp")
project2 = cw.open_project("traces/f303_mbedtls_x4_key0.cwp")
project3 = cw.open_project("/mnt/hgfs/trace_key_6000/key_0.cwp")

print(project1.location)
xShape = project1.traces[0].wave.shape[0]
x=np.linspace(0,xShape-1, xShape)
#print(x)
colorList = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00"]

fig = go.Figure()
fig = make_subplots(rows=1, cols=1, shared_yaxes=True, shared_xaxes=True, \
)
for idx in range(5):
    fig.add_trace(go.Line(x=x, y=project1.traces[idx].wave, line=dict(color=colorList[0]), name= "f303_10kAdd"), row=1, col=1)
    fig.add_trace(go.Line(x=x, y=project2.traces[idx].wave, line=dict(color=colorList[1]), name= "newTrace"), row=1, col=1)
    fig.add_trace(go.Line(x=x, y=project3.traces[idx].wave, line=dict(color=colorList[2]), name= "f303_orig"), row=1, col=1)

fig.update_layout(
    xaxis1_title="Time",
    yaxis1_title="Power",
    showlegend=True
  )

savePath = "img/f303_comapre_old_new_6000_traces.html"
fig.write_html(savePath, auto_open=True)


/mnt/hgfs/trace_key_f303_10kAdd


/home/manoj/.pyenv/versions/cw/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




# Compare F415: 5WS vs 0WS vs 30db

In [9]:
project1 = cw.open_project("/home/manoj/Documents/chipwhisperer/jupyter/courses/sca201/traces/f415_mbedtls_1500.cwp")
project2 = cw.open_project("/home/manoj/Documents/chipwhisperer/jupyter/courses/sca201_modified/traces/f415_mbedtls_1500_latency0.cwp")
project3 = cw.open_project("/home/manoj/Documents/chipwhisperer/jupyter/courses/sca201_modified/traces/f415_mbedtls_1500_latency0_30db.cwp")
project4 = cw.open_project("/mnt/hgfs/trace_key_f4/key_0.cwp")

print(project1.location)
xShape = project1.traces[0].wave.shape[0]
x=np.linspace(0,xShape-1, xShape)
#print(x)
colorList = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00"]

fig = go.Figure()
fig = make_subplots(rows=1, cols=1, shared_yaxes=True, shared_xaxes=True, \
)
for idx in range(5):
    fig.add_trace(go.Line(x=x, y=project1.traces[idx].wave, line=dict(color=colorList[0]), name= "f415"), row=1, col=1)
    fig.add_trace(go.Line(x=x, y=project2.traces[idx].wave, line=dict(color=colorList[1]), name= "f415_0lat"), row=1, col=1)
    fig.add_trace(go.Line(x=x, y=project3.traces[idx].wave, line=dict(color=colorList[2]), name= "f415_0lat_30db"), row=1, col=1)
    fig.add_trace(go.Line(x=x, y=project4.traces[idx].wave, line=dict(color=colorList[3]), name= "f415_orig"), row=1, col=1)

fig.update_layout(
    xaxis1_title="Time",
    yaxis1_title="Power",
    showlegend=True
  )

savePath = "img/f415_5ws_0ws_30db_trace_compare.html"
fig.write_html(savePath, auto_open=True)

/home/manoj/Documents/chipwhisperer/jupyter/courses/sca201/traces


 # Compare traces for key=0 across all devices

In [53]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, scale, minmax_scale, robust_scale


def convert2numpy(prj):
    traces = []
    for idx, trace in enumerate(prj.traces):
        traces.append(trace.wave)  
    traces = np.array(traces)
    return traces

def preProcessing(x, preType):
    if(preType == "stdScaler"):
        sc = StandardScaler()
        x = sc.fit_transform(x)
    elif(preType=="minMax"):
        sc = MinMaxScaler()
        x = sc.fit_transform(x)
    elif(preType=="robust"):
        sc = RobustScaler()
        x = sc.fit_transform(x)
        
    return x 

def preProcessing_row(x, preType):
    if(preType == "stdScaler"):
        x = scale(x, axis=1)
    elif(preType=="minMax"):
        sc = MinMaxScaler()
        x = minmax_scale(x, axis=1, feature_range=(-1, 1),)
    elif(preType=="robust"):
        x = robust_scale(x, axis=1)
        
    return x
        
project1 = cw.open_project("/mnt/hgfs/trace_key_f303_1500/key_0.cwp")
project2 = cw.open_project("/mnt/hgfs/trace_key_l443_7p5kAdd/key_0.cwp")
#project3 = cw.open_project("/mnt/hgfs/trace_key_f4/key_0.cwp")
project_3 = cw.open_project("/home/manoj/Documents/chipwhisperer/jupyter/courses/sca201_modified/traces/f415_mbedtls_1500_latency0.cwp")
project4 = cw.open_project("/home/manoj/Documents/chipwhisperer/jupyter/courses/sca201_modified/traces/f415_mbedtls_1500_latency0_30db.cwp")

print(project1.location)
xShape = project1.traces[0].wave.shape[0]
x=np.linspace(0,xShape-1, xShape)
#print(x)
colorList = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00"]

f303 = convert2numpy(project1)
l443 = convert2numpy(project2)
f415 = convert2numpy(project3)
f415_lat0_30db = convert2numpy(project4)

fig = go.Figure()
fig = make_subplots(rows=4, cols=2, shared_yaxes=True, shared_xaxes=True, \
)
for idx in range(1):
    fig.add_trace(go.Line(x=x, y=f303[idx], line=dict(color=colorList[0]), name= "f303"), row=1, col=1)
    fig.add_trace(go.Line(x=x, y=l443[idx], line=dict(color=colorList[1]), name= "l443"), row=1, col=1)
    fig.add_trace(go.Line(x=x, y=f415[idx], line=dict(color=colorList[2]), name= "f415"), row=1, col=1)
    fig.add_trace(go.Line(x=x, y=f415_lat0_30db[idx], line=dict(color=colorList[3]), name= "f415_lat0_30db"), row=1, col=1)
    
    fig.add_trace(go.Line(x=x, y=preProcessing(f303, "stdScaler")[idx], line=dict(color=colorList[0]), name= "f303"), row=2, col=1)
    fig.add_trace(go.Line(x=x, y=preProcessing(l443, "stdScaler")[idx], line=dict(color=colorList[1]), name= "l443"), row=2, col=1)
    fig.add_trace(go.Line(x=x, y=preProcessing(f415, "stdScaler")[idx], line=dict(color=colorList[2]), name= "f415"), row=2, col=1)
    fig.add_trace(go.Line(x=x, y=preProcessing(f415_lat0_30db, "stdScaler")[idx], line=dict(color=colorList[3]), name= "f415_lat0_30db"), row=2, col=1)
    
    fig.add_trace(go.Line(x=x, y=preProcessing(f303, "minMax")[idx], line=dict(color=colorList[0]), name= "f303"), row=3, col=1)
    fig.add_trace(go.Line(x=x, y=preProcessing(l443, "minMax")[idx], line=dict(color=colorList[1]), name= "l443"), row=3, col=1)
    fig.add_trace(go.Line(x=x, y=preProcessing(f415, "minMax")[idx], line=dict(color=colorList[2]), name= "f415"), row=3, col=1)
    fig.add_trace(go.Line(x=x, y=preProcessing(f415_lat0_30db, "minMax")[idx], line=dict(color=colorList[3]), name= "f415_lat0_30db"), row=3, col=1)
    
    fig.add_trace(go.Line(x=x, y=preProcessing(f303, "robust")[idx], line=dict(color=colorList[0]), name= "f303"), row=4, col=1)
    fig.add_trace(go.Line(x=x, y=preProcessing(l443, "robust")[idx], line=dict(color=colorList[1]), name= "l443"), row=4, col=1)
    fig.add_trace(go.Line(x=x, y=preProcessing(f415, "robust")[idx], line=dict(color=colorList[2]), name= "f415"), row=4, col=1)
    fig.add_trace(go.Line(x=x, y=preProcessing(f415_lat0_30db, "robust")[idx], line=dict(color=colorList[3]), name= "f415_lat0_30db"), row=4, col=1)
    
    fig.add_trace(go.Line(x=x, y=preProcessing_row(f303, "stdScaler")[idx], line=dict(color=colorList[0]), name= "f303"), row=2, col=2)
    fig.add_trace(go.Line(x=x, y=preProcessing_row(l443, "stdScaler")[idx], line=dict(color=colorList[1]), name= "l443"), row=2, col=2)
    fig.add_trace(go.Line(x=x, y=preProcessing_row(f415, "stdScaler")[idx], line=dict(color=colorList[2]), name= "f415"), row=2, col=2)
    fig.add_trace(go.Line(x=x, y=preProcessing_row(f415_lat0_30db, "stdScaler")[idx], line=dict(color=colorList[3]), name= "f415_lat0_30db"), row=2, col=2)
    
    fig.add_trace(go.Line(x=x, y=preProcessing_row(f303, "minMax")[idx], line=dict(color=colorList[0]), name= "f303"), row=3, col=2)
    fig.add_trace(go.Line(x=x, y=preProcessing_row(l443, "minMax")[idx], line=dict(color=colorList[1]), name= "l443"), row=3, col=2)
    fig.add_trace(go.Line(x=x, y=preProcessing_row(f415, "minMax")[idx], line=dict(color=colorList[2]), name= "f415"), row=3, col=2)
    fig.add_trace(go.Line(x=x, y=preProcessing_row(f415_lat0_30db, "minMax")[idx], line=dict(color=colorList[3]), name= "f415_lat0_30db"), row=3, col=2)
    
    fig.add_trace(go.Line(x=x, y=preProcessing_row(f303, "robust")[idx], line=dict(color=colorList[0]), name= "f303"), row=4, col=2)
    fig.add_trace(go.Line(x=x, y=preProcessing_row(l443, "robust")[idx], line=dict(color=colorList[1]), name= "l443"), row=4, col=2)
    fig.add_trace(go.Line(x=x, y=preProcessing_row(f415, "robust")[idx], line=dict(color=colorList[2]), name= "f415"), row=4, col=2)
    fig.add_trace(go.Line(x=x, y=preProcessing_row(f415_lat0_30db, "robust")[idx], line=dict(color=colorList[3]), name= "f415_lat0_30db"), row=4, col=2)
    
    
fig.update_layout(
    xaxis1_title="Time",
    yaxis1_title="Power",
    showlegend=True
  )

savePath = "img/f303_l443_f415_comparison.html"
fig.write_html(savePath, auto_open=True)

/mnt/hgfs/trace_key_f303_1500


/home/manoj/.pyenv/versions/cw/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/manoj/.pyenv/versions/cw/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/manoj/.pyenv/versions/cw/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/manoj/.pyenv/versions/cw/lib/python3.9/site-packages/plotly/graph_objs

# One device different pre-processing

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, scale, minmax_scale, robust_scale


def convert2numpy(prj):
    traces = []
    for idx, trace in enumerate(prj.traces):
        traces.append(trace.wave)  
    traces = np.array(traces)
    return traces

def preProcessing(x, preType):
    if(preType == "stdScaler"):
        sc = StandardScaler()
        x = sc.fit_transform(x)
    elif(preType=="minMax"):
        sc = MinMaxScaler()
        x = sc.fit_transform(x)
    elif(preType=="robust"):
        sc = RobustScaler()
        x = sc.fit_transform(x)        
    return x 

def preProcessing_row(x, preType):
    if(preType == "stdScaler"):
        x = scale(x, axis=1)
    elif(preType=="minMax"):
        sc = MinMaxScaler()
        x = minmax_scale(x, axis=1, feature_range=(-1, 1),)
    elif(preType=="robust"):
        x = robust_scale(x, axis=1)        
    return x


In [31]:
import numpy as np
project1 = cw.open_project("/mnt/hgfs/trace_key_f303_1500/key_0.cwp")
print(dir(project1.traces))
traces = []

for idx, trace in enumerate(project1.traces):
    traces.append(trace.wave)
    
traces = np.array(traces)
print(traces)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'append', 'cur_seg', 'cur_trace_num', 'extend', 'keys', 'max', 'project', 'seg_ind_max', 'seg_len', 'tm']
[[0.05859375 0.05859375 0.04589844 ... 0.01757812 0.07421875 0.09179688]
 [0.06152344 0.05761719 0.04492188 ... 0.01660156 0.07617188 0.09375   ]
 [0.06152344 0.06152344 0.046875   ... 0.01855469 0.07324219 0.09472656]
 ...
 [0.06152344 0.06054688 0.04882812 ... 0.01953125 0.07714844 0.09765625]
 [0.0625     0.06054688 0.04785156 ... 0.01953125 0.07714844 0.09472656]
 [0.0625     0.06054688 0.04882812 ... 0.02050781 0.07617188 0.09570312]]


In [42]:
type(traces)

numpy.ndarray

In [ ]:
!pip install PyWavelets

In [13]:
import pywt

project = cw.open_project("traces/l443_tinyAES_x4_4ktime_key0.cwp")

scales = range(1,21)
waveletName = "gaus5"
coeff0, freq0 = pywt.cwt(project.traces[0].wave, scales, waveletName, 1)
coeff1, freq1 = pywt.cwt(project.traces[1].wave, scales, waveletName, 1)
coeff2, freq2 = pywt.cwt(project.traces[2].wave, scales, waveletName, 1)

x = np.linspace(0, coeff0.shape[1]-1, coeff0.shape[1])
colorList = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00"]

fig = go.Figure()

fig = make_subplots(rows=6, cols=1, shared_yaxes=True, shared_xaxes=True,
)
for idx in range(6):
    if(idx==0):
        fig.add_trace(go.Line(x=x, y=project.traces[idx].wave, line=dict(color=colorList[3])), row=idx+1, col=1)
    else:
        fig.add_trace(go.Line(x=x, y=coeff0[idx], line=dict(color=colorList[0])), row=idx+1, col=1)
        fig.add_trace(go.Line(x=x, y=coeff1[idx], line=dict(color=colorList[1])), row=idx+1, col=1)
        fig.add_trace(go.Line(x=x, y=coeff2[idx], line=dict(color=colorList[2])), row=idx+1, col=1)

fig.update_layout(
#   xaxis2_title="Time",
#   yaxis1_title="Power",
#   yaxis2_title="Power",
    showlegend=False
    )

savePath = "img/wavelet_l443_tinyAES_x4_4ktime_key0_x4_gaus5.html"
fig.write_html(savePath, auto_open=True)


/home/manoj/.pyenv/versions/cw/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


